In [ ]:
import copy
from tqdm.auto import tqdm
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt


from superprimitive_fusion.scanner import (
    virtual_mesh_scan,
    mesh_depth_image,
    generate_rgbd_noise,
    clean_mesh_and_remap_weights,
)
from superprimitive_fusion.utils import (
    bake_uv_to_vertex_colours,
    polar2cartesian,
    distinct_colours,
)
from superprimitive_fusion.mesh_fusion import (
    fuse_meshes,
)
from superprimitive_fusion.mesh_fusion_utils import (
    get_mesh_components,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
names = (
    ('mustard-bottle', 'mustard-bottle.obj'),
    ('table', 'table.obj'),
    ('power-drill', 'power-drill.obj'),
    ('bleach', 'bleach.obj'),
    ('pitcher', 'pitcher.obj'),
    ('mug', 'mug.obj'),
    ('extra-large-clamp', 'extra-large-clamp-leaning.obj'),
)

gt_meshes = dict()
for foldername,filename in names:
    print(f'Getting the {foldername}')
    
    gt_mesh = o3d.io.read_triangle_mesh(f"../data/posed-meshes/{foldername}/{filename}", enable_post_processing=True)

    bake_uv_to_vertex_colours(gt_mesh)
    gt_mesh.compute_vertex_normals()

    gt_meshes[foldername] = gt_mesh

gt_mesh_list = list(gt_meshes.values())

Getting the mustard-bottle
Getting the table
Getting the power-drill
Getting the bleach
Getting the pitcher
Getting the mug
Getting the extra-large-clamp


In [3]:
centres = []
for gt_meshname, gt_mesh in gt_meshes.items():
    if gt_meshname == 'table':
        continue
    centres.append(gt_mesh.get_center())

centres = np.vstack(centres)

obj_centre = centres.mean(axis=0)

In [4]:
cam_centre_offset=np.array([0, 0, 0.2])
look_at = obj_centre
width_px: int = 360
height_px: int = 240
fov: float = 70.0
k: float = 3.5
max_normal_angle_deg = None

In [97]:
scans = []
N = 10
cam_centres = []
for theta in tqdm(np.linspace(0,360/N * (N-1), N), desc='Scanning'):
# for theta in np.linspace(30,180, N):
# for theta in [70, 110]:
    cc = cam_centre_offset + polar2cartesian(0.8, 30, theta)
    cam_centres.append(cc)

    object_meshes, object_weights = virtual_mesh_scan(
        meshlist=gt_mesh_list,
        cam_centre=cc,
        look_at=look_at,
        k=10,
        max_normal_angle_deg=None,
        linear_depth_sigma=2.5e-4,
        quadrt_depth_sigma=2.5e-4,
        sigma_floor=1e-4,
        bias_k1=0.03,
        # linear_depth_sigma=0,
        # quadrt_depth_sigma=0,
        # sigma_floor=0,
        # bias_k1=0,
    )
    scans.append([(m,w) for m,w in zip(object_meshes, object_weights)])

meshlists = [[scan_obj[0] for scan_obj in scan] for scan in scans]
meshes = np.array(meshlists).flatten().tolist()

Scanning:   0%|          | 0/10 [00:00<?, ?it/s]

/home/christopher/Documents/Code/project/src/superprimitive_fusion/scanner.py:560: RuntimeWarning: invalid value encountered in matmul
  z = ((points - cam_centre) @ L).clip(min=0.0) # (H,W)


In [109]:
# DEMO mesh scan colours
diff_mesh_colours = distinct_colours(len(meshes))
diff_col_meshes = []
for i,mesh in enumerate(meshes):
    if len(np.asarray(mesh.vertices)) == 0:
        continue
    col_mesh = copy.deepcopy(mesh)
    col_mesh.compute_vertex_normals()
    col_mesh.paint_uniform_color(diff_mesh_colours[i])
    diff_col_meshes.append(col_mesh)

meshes_nrm = copy.deepcopy(meshes)
for mesh in meshes_nrm:
    if len(np.asarray(mesh.vertices)) == 0:
        continue
    mesh.compute_vertex_normals()

front = look_at - cam_centre_offset + polar2cartesian(0.8, 110, 90)
front /= np.linalg.norm(front)
o3d.visualization.draw_geometries(
    geometry_list=diff_col_meshes,
    lookat=look_at,
    front=front,
    up=np.array([0, 0, 1]),
    zoom=0.11,
)

In [ ]:
fused_scan = scans[0]
N = len(scans[0])
assert all(len(s) == N for s in scans), "All scans must have the same length"

for t, scan in enumerate(tqdm(scans[1:], desc="Scan", unit="scan"), start=1):
    fused_scan_meshes = []
    fused_scan_weights = []
    for obj_id in tqdm(range(N), desc="Object", unit="obj", position=1, leave=False):
        mesh1, weights1 = fused_scan[obj_id]
        mesh2, weights2 = scan[obj_id]

        if len(np.asarray(mesh1.vertices)) == 0:
            tqdm.write(f"Scan {t-1} obj {obj_id} is empty")
            fused_obj_mesh, fused_obj_weights = mesh2, weights2
        elif len(np.asarray(mesh2.vertices)) == 0:
            tqdm.write(f"Scan {t} obj {obj_id} is empty")
            fused_obj_mesh, fused_obj_weights = mesh1, weights1
        else:
            fused_obj_mesh, fused_obj_weights = fuse_meshes(
                mesh1=mesh1, weights1=weights1,
                mesh2=mesh2, weights2=weights2,
                h_alpha=5.0, r_alpha=2.0,
                nrm_shift_iters=3, nrm_smth_iters=1,
                shift_all=False, fill_holes=False,
            )
        fused_scan_meshes.append(fused_obj_mesh)
        fused_scan_weights.append(fused_obj_weights)

    fused_scan = list(zip(fused_scan_meshes, fused_scan_weights))


Scan:   0%|          | 0/9 [00:00<?, ?scan/s]

Object:   0%|          | 0/7 [00:00<?, ?obj/s]

Scan 0 obj 0 is empty


Object:   0%|          | 0/7 [00:00<?, ?obj/s]

Object:   0%|          | 0/7 [00:00<?, ?obj/s]

...

Object:   0%|          | 0/7 [00:00<?, ?obj/s]

Scan 7 obj 3 is empty
Scan 7 obj 5 is empty


Object:   0%|          | 0/7 [00:00<?, ?obj/s]

Scan 8 obj 0 is empty


Object:   0%|          | 0/7 [00:00<?, ?obj/s]

In [ ]:
fused_mesh_result = [obj[0] for obj in fused_scan]

In [107]:
diff_mesh_colours = distinct_colours(len(fused_mesh_result))
diff_col_meshes = []
for i,mesh in enumerate(fused_mesh_result):
    if len(np.asarray(mesh.vertices)) == 0:
        continue
    col_mesh = copy.deepcopy(mesh)
    col_mesh.compute_vertex_normals()
    col_mesh.paint_uniform_color(diff_mesh_colours[i])
    diff_col_meshes.append(col_mesh)

front = look_at - cam_centre_offset + polar2cartesian(0.8, 110, 90)
front /= np.linalg.norm(front)
o3d.visualization.draw_geometries(
    geometry_list=fused_mesh_result,
    lookat=look_at,
    front=front,
    up=np.array([0, 0, 1]),
    zoom=0.15,
)